# Scenario Analysis: Measles Interventions

This notebook demonstrates how to use the `create_scenario` API to model different intervention strategies:
1. **Baseline**: Current forecast.
2. **Increased Vaccination**: Reducing transmission ($\beta$).
3. **Border Control**: Reducing importation ($\epsilon$).

In [ ]:
import pandas as pd
import numpy as np
import logging
from epydemics import DataContainer, Model
from epydemics.analysis.visualization import compare_scenarios

logging.basicConfig(level=logging.INFO)

## 1. Setup Data (Synthetic for Demo)

In [ ]:
# Create synthetic incidence data (sporadic outbreaks, 25 years)
dates = pd.date_range(start="2000-01-01", periods=25, freq="YE")
# Generate random sporadic data
np.random.seed(42)
I = (np.random.poisson(5, 25) * np.random.choice([0, 1, 10], 25, p=[0.6, 0.3, 0.1])).astype(float)
data = pd.DataFrame({
    "I": I,
    "N": [1000.0] * 25,
    "D": [0.0] * 25
}, index=dates)

container = DataContainer(data, mode="incidence")

## 2. Fit Baseline Model

In [ ]:
model = Model(container, importation_rate=0.01) # Baseline importation
model.create_model()
model.fit_model(max_lag=1)
model.forecast(steps=5)
model.run_simulations(n_jobs=1)
model.generate_result()

## 3. Create Scenarios

In [ ]:
# Scenario 1: Aggressive Vaccination (Reduce Beta by 50%)
results_vax = model.create_scenario("Aggressive Vaccination", {'beta': 0.5})

# Scenario 2: Strict Border Control (No Importation)
results_border = model.create_scenario("Closed Borders", {'importation_rate': 0.0})

# Scenario 3: High Transmission Risk (Beta + 20%)
results_risk = model.create_scenario("High Risk", {'beta': 1.2})

## 4. Compare Scenarios

In [ ]:
scenarios = {
    "Baseline": model.results,
    "Vaccination (-50% Beta)": results_vax,
    "Closed Borders (No Import)": results_border,
    "High Risk (+20% Beta)": results_risk
}

compare_scenarios(scenarios, "I", title="Impact of Interventions on Incident Cases")